In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests

ua = UserAgent()

In [2]:
headers = {
'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
'cache-control': 'max-age=0',
'cookie': 'perseusRolloutSplit=2; dhhPerseusSessionId=1572777846270.344129640382112400.tqve6035lc8; dhhPerseusGuestId=1572773675054.296431960063008960.th00zfvm5m; __cfduid=d49830dd8faef655d8fd7859443a1fd1a1572773671; hl=en; AppVersion=02c2d36; ld_key=118.100.76.150; _pxhd=6097c3582e3b323a10236581cc2f09942a843225042959a6592317fbc32efa2c:23bbaec1-fe1d-11e9-b8fd-5d9da8797a65; optimizelyEndUserId=oeu1572773673643r0.3029206072281285; _pxvid=23bbaec1-fe1d-11e9-b8fd-5d9da8797a65; perseusRolloutSplit=2; dhhPerseusGuestId=1572773675054.296431960063008960.th00zfvm5m; _gcl_au=1.1.1974466263.1572773675; _ga=GA1.2.834634854.1572773675; _gid=GA1.2.733629929.1572773675; _hjid=cbf68d6c-0e9a-4343-87da-bdb6dd825dc6; __ssid=b43de24b73336ad349e495d33424f8e; ViewContentStatus=true; _dc_gtm_UA-90537345-1=1; _dc_gtm_UA-50811932-30=1; _px3=3c729ac794d8d45c7fe754cbd67790d71474a785a2627fe57b8dd1354665be4e:fNXD1m3oMRkZmH5yzruS63pfq2wij+y7tLFTLJITIoz/x88BJDR5YkmH2xWELbrnp0cfUmp+8OXMnJIjFHB8Qw==:1000:AJqIWGYwqD+LpejoGvA2HPJbAfvzQ+jAwi0RQCe+zvgkpfJaiErJeIMGRrPiJG7PVkepT8hmntejklhnnxF8qR57jOzmSCAwb5kSaOuaqH2+u7jScpM+KnlZgmozgyUaqj6QccbWsJ1FO2bNoyzvj//TsuWcVj6EeyYubyxqnQ4=; _tq_id.TV-81365445-1.cf54=fc76d66efbbf4a71.1572773676.0.1572779104..',
'sec-fetch-mode': 'navigate',
'sec-fetch-site': 'none',
'sec-fetch-user': '?1',
'upgrade-insecure-requests': '1',
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}

In [3]:
# headers = {"User-Agent": ua.chrome, 
#            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
#            "Accept-Language": "en-US,en;q=0.5", 
#            "Accept-Encoding": "gzip, deflate", "DNT": "1", "Connection": "close", 
#            "Upgrade-Insecure-Requests": "13434"}
cookies = {'JSESSIONID': 'fbadab78eef7d2d'}

In [4]:
import json

with open('foodpanda-restaurant.json') as fopen:
    restaurants = json.load(fopen)
    
restaurants.keys()

dict_keys(['Kuala Lumpur', 'Penang', 'Petaling Jaya', 'Subang', 'Shah Alam', 'Cyberjaya', 'Johor Bahru', 'Melaka', 'Kuching', 'Miri', 'Kuantan', 'Bangi', 'Ipoh', 'Terengganu', 'Kajang', 'Kelantan', 'Putrajaya', 'Kota-Kinabalu', 'Klang', 'Negeri Sembilan', 'Sungai Petani', 'Alor Setar', 'Kulim', 'Rawang', 'Taiping', 'Sitiawan', 'Kluang', 'Kulai', 'Sibu', 'Sepang', 'Kangar', 'Temerloh', 'Bintulu', 'Petra Jaya', 'Sandakan', 'Pontian', 'Segamat', 'Nibong Tebal', 'Kampar', 'Ulu Tiram', 'Kepala Batas', 'Jitra', 'Port Dickson', 'Teluk Intan', 'Arau', 'Jerantut', 'Bahau', 'Mersing', 'Kota Tinggi', 'Rompin', 'Kerian', 'Kuala Pilah', 'Kuala Kangsar', 'Muallim', 'Samarahan', 'Tangkak', 'Seri Iskandar'])

In [5]:
restaurants['Kuala Lumpur']['Q Bistro Nasi Kandar']

{'star': '4.4',
 'delivery': 'Free',
 'characters': ['Noodles',
  'Rice',
  'Local Champions',
  'foodpanda Favourites',
  'Exclusive on foodpanda',
  'Malaysian food'],
 'link': '/chain/cm1zm/q-bistro-nasi-kandar'}

In [6]:
import time

parsed = {}

In [7]:
quit = False

for no, (city, city_values) in enumerate(list(restaurants.items())):
    if quit:
        break
        
    for no_nested, (restaurant, restaurant_values) in enumerate(list(city_values.items())):
        if '%s <> %s'%(city, restaurant) in parsed:
            print('%s <> %s'%(city, restaurant), ' already inside parsed')
            continue
    
        try:
            response = requests.get('https://www.foodpanda.my' + restaurant_values['link'], 
                        headers = headers, cookies = cookies, timeout = 60)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            if 'Access to this page has been denied' in soup.title.contents[0]:
                quit = True
                print('quit at ', no, no_nested)
                break
                
            foods = soup.find_all("div", class_="desktop-cart mobile-cart__hidden")
            
            print('PARSING ', restaurant)
            data = []
            for food in foods:
                try:
                    data.append(json.loads(food.get('data-vendor')))
                except:
                    print('ERROR 404 food ', restaurant)
            parsed['%s <> %s'%(city, restaurant)] = data
            restaurants[city][restaurant]['data'] = data
            
            
        except:     
            print('ERROR 404 ', restaurant)

PARSING  Q Bistro Nasi Kandar
PARSING  Nasi Lemak Famous
PARSING  WILD
PARSING  Idli Only Cafe
PARSING  The Butter Chicken
PARSING  SpiceWays
PARSING  Restoran Fathima
PARSING  Nasi Kandar Utara
PARSING  Wok the Wok
PARSING  Zoe Bangsar
PARSING  My Halal Snack Pack @ Bangsar
PARSING  Mama Mia Mapo Tofu
PARSING  Nemo's Nachos @ Bangsar
PARSING  JJ Crab Omelette @ Bangsar
PARSING  Pasta Pedas
PARSING  My Taiwan Secret Braised Chicken @ Bangsar
PARSING  KimKim @ Bangsar
PARSING  Take5 Jazz Café
PARSING  Saravana  Mess
PARSING  Nawab's Briyani
PARSING  HYDERABAD BIRYANI PALACE
PARSING  Restoran Chat Masala (pure vegetarian)
PARSING  The Treats
PARSING  Nook, Aloft KL Sentral
PARSING  Legend's Claypot Briyani
PARSING  KFC
PARSING  Jardin Coffee & Tea
PARSING  Vegetarian Stall
PARSING  Daporlah
PARSING  Original Penang Kayu Nasi Kandar
PARSING  Chizu
PARSING  The Good Rice Bowls
PARSING  Gajah Mada (Nu Sentral)
PARSING  QUIN-WOW!
PARSING  Peranakan Place (Nu Sentral)
PARSING  Auntie Anne's
P

PARSING  Restro Uwong Palembang
PARSING  Indian Palace
PARSING  Sakurajima Japanese & Chinese Restaurant @Penang Golf Club
PARSING  Western Food (Giant Food Court Bayan Baru)
PARSING  KFC
PARSING  Simply Leafy
PARSING  Hot Papa Corner
PARSING  Ina Mee Sotong
PARSING  Cucur Udang Pasar Bayan Baru
PARSING  Ming Xiang Tai
PARSING   Bonda Coconut Shake
PARSING  East meets west set rice
PARSING  Asamu Taiwan Snack
PARSING  Blok5 Food.Drink
PARSING  Ah Lai Curry Mee
PARSING  Xiao Lin Hokkien Mee
PARSING  Wu Ming Apom
PARSING  femes coconut shake
PARSING  E-Lissa Burger Green Mayo
PARSING  Restoran Pen Mutiara Batu Maung
PARSING  Bayan Baru Koay Teow Thng
PARSING  Simply Sedap Cafe & Restaurant Relau
PARSING  Project Banana Cheese
PARSING  Marrybrown
PARSING  Ayam Jingkrak Mbax Ardilla (Bukit Jambul Complex)
PARSING  Aris Cheese Naan (Gelugor)
PARSING  SERAI THAI
PARSING  FBCCM KITCHEN & Co
PARSING  Subway
PARSING  Black Sugar Dynasty
PARSING  Penang Fried Chicken Western Food
PARSING  Korean

PARSING  Sanyoso Mazesoba (Kota Damansara)
PARSING  Neighbourhood Food court
PARSING  Man Bei
PARSING  7686 Choy Kor Homecook Recipes
PARSING  Wong Kee (Restaurant D.U. Cafe)
PARSING  Lérz’ 
PARSING  GOGIGO (KOTA DAMANSARA) KOREAN RESTAURANT
PARSING  CHILLI (Tropicana)
PARSING  Happy Tummy Dessert
PARSING  Hakka Rang Restaurant 客家人饭店
PARSING  TCW Kitchen and Bar
PARSING  Ben Gong's Tea
PARSING  BG Noodle House
PARSING  Sushi Ya
PARSING  Uncle Lee Chicken Rice
PARSING  MJ Cafe & Restaurant
PARSING  Papparich
PARSING  Vege Harbour
PARSING  TL Super Pot
PARSING  Uncle Don's
PARSING  Ayn ZamZam Restaurant
PARSING  Summer Breeze Cafe
PARSING  Uncle Crab Kota Damansara
PARSING  Baywatch Bistro
PARSING  THE CHIEF Cuisine & Brew
PARSING  Subway
PARSING  Burger King
PARSING  Lavo
PARSING  Roti Bakery & Cafe
PARSING  Drink Station (Kota Damansara)
PARSING  Hey! I am Yogost
PARSING  Aoki-Tei Japanese Restaurant
PARSING  Eee sparkling
PARSING  DoDoFuk Old Noodle House 多多福老麵館
PARSING  pin tea
PARSI

PARSING  JW Recipe @ Restoran Soon Sing
PARSING  Kampung Chicken Rice & Noodle @ Restoran 268 Soon Sing
PARSING  Pork Noodle @ Restoran 268 Soon Sing
PARSING  Food Indonesia Bandung @ 928 Foodcourt
PARSING  Shelove Waffle Cafe
PARSING  Restoran Kiat Seng
PARSING  Hou Kee Pork Noodle @ Kopitiam 95 Pusat Bandar Puchong
PARSING  Monster Pig
PARSING  Sushi ZenS (Puchong)
PARSING  Gombak Hokkien Mee
PARSING  Hometown Fish Restaurant
PARSING  Face To Face Noodle House
PARSING  SUP MELETOP
PARSING  Moon Palace
PARSING  Hometown Hainan Coffee
PARSING  Brother Kuan
PARSING  Restoran Maju Utama (Puchong)
PARSING  Mr Chicken Restaurant
PARSING  Restoran Mun Ji Chop Noodle Zone (Bandar Puteri Puchong)
PARSING  Satay & Katering Mak Mah
PARSING  A Taste of Yi Cafe @ Wash & Blow
PARSING  Souliquid
PARSING  Koong Woh Tong
PARSING  Brilliant Nasi Lemak
PARSING  The Wok
PARSING  SRK Noodle House
PARSING  Good Taste Restaurant
PARSING  Gangnam88 Puchong Branch
PARSING  Aneka Mee @ tesco extra pusat banda

PARSING  Ayam Geprek Masdu
PARSING  Tealive (NSK Kota Damansara)
PARSING  Pak Hailam Kopitiam
PARSING  Dazs Donut & Coffee (Tesco Extra)
PARSING  MYLAKSA Tesco Extra Shah Alam
PARSING  The Chicken Rice Shop
PARSING  ARR
PARSING  Level One Kitchen
PARSING  NIESHA FUN FOOD
PARSING  Bask Bear Coffee
PARSING  Aydan’s Kitchen
PARSING  Mimi Tomyam Kelapa
PARSING  Nasi Dulang Pyorr Bukit Jelutong
PARSING  King Grill
PARSING  Pabloz Cafe
PARSING  Kapten Malaya Cafe
PARSING  Restoran Canton Village
PARSING  Soi55 Thai Kitchen & Bar
PARSING  Burger Al Falah
PARSING  Nasi Kandar Zubair Bistro
PARSING  Panmour Villa Shah Alam
PARSING  Burger Periuk
PARSING  Ali's Corner Seksyen 20
PARSING  Mr Bubur
PARSING  D'SURIE (Restoran Lenggong)
PARSING  Khulafa
PARSING  Bintang Irfan maju
PARSING  Grease
PARSING  Restoran MMZ Bukit Jelutong
PARSING  Bro Burger
PARSING  Laman Grill Steak & Bar-B-Que - Shah Alam
PARSING  Polperro Steak House (Seksyen 13)
PARSING  Nasi Lemak Cinta Sayang (Seksyen 13)
PARSING  

...

PARSING  Mr. King Squid
PARSING  Bread History @ Gurney Plaza
PARSING  Hye Mummy's
PARSING  Asraar Hadramawt Restaurant
PARSING  DS Kitchen
PARSING  Laksa Penang Piaww @ Mydin Bertam
PARSING  Restoran Thaiyyub Nasi Kandar
PARSING  Happy Waffle
PARSING  Bertam Corner Restaurants
PARSING  Bismie Murtabak Cheese House
PARSING  Nasi Kandar Mesra Maju Mydin Mall
PARSING  D Faez Bahulu
PARSING  Nasi Ayam @ Mydin Bertam
PARSING  Dr Donuts Mydin Bertam
PARSING  Swarma @ Mydin Bertam
PARSING  The Kebuloque Truck
PARSING  OhhDen Western
PARSING  OhhDen Cafe
PARSING  D'iena Bakery & Cafe
PARSING  acocafe
PARSING  Han's Hadramawt (Putera Bertam)
PARSING  Putra Seafood Cafe
PARSING  Asuwa D Sayang
PARSING  Keropok Lekor Kuala Besut - Taiping Sentral
PARSING  ACU Burger
PARSING  Burger Cabin
PARSING  Acu Street Burger
PARSING  Elfinia Cafe(Big Screen)
PARSING  86 Char Ho Fun (union Kedai Kopi Dan Makanan)
PARSING  Sally Cafe
PARSING  老嫲子客家卤肉饭 Wang Lai Cafe
PARSING  Restoran Maheran
PARSING  Restoran

PARSING  AA MAMA STATION (STALL MEE JAWA MAK SITI)
PARSING  Marie's corndog
PARSING  NISS COFFEE
PARSING  pd sha rest
PARSING  restoran habsah tomyam
PARSING  Fatimah Bakery
PARSING  King's Char Kuey Teow
PARSING  Kaidz Cafe
PARSING  Sup Kedah Pak Su Naim PD
PARSING  kay dees burger
PARSING  Pecal Lele & Ayam Penyet Tomyam Seafood
PARSING  Bagan Char Keow Tiow
PARSING  warung roti canai sedap
PARSING  Nasi Ayam Panggang Beratoq
PARSING  Rahang Fishball Noodle
PARSING  Norainee Adam
PARSING  Gerai Kantin MPPD Kakak Sayang
PARSING  Xiwang Seafood Village Restaurant
PARSING  Tropiko Café & Bakery
PARSING  original paklang kopitiam
PARSING  Kafe Kampung Kaw
PARSING  Kepak Madu Batu 4
PARSING  Krabby Patty
PARSING  Deli2go & Costa Coffee
PARSING  Warung d'corner
PARSING  Takoyaki & Oden
PARSING  Kedai Kopi Tian Xiang Cheng Ipoh White Coffee (天香城怡保白咖啡)
PARSING  Gerai Orisa
PARSING  Hads Kitchen (Pisang Cheese Crispy)
PARSING  Restoran A. Ajmal (Kg. Banjar)
PARSING  Restoran Al-Maaidah
PARSIN

PARSING  Rasa Sayang
PARSING  Restoran Zulya
PARSING  Auntie Elaine Soya Bean @ Mersing
PARSING  Padi Emas Kitchen
PARSING  Kedai Makan Siti
PARSING  Cakemania Cafe
PARSING  Greenland Cafe
PARSING  Restoran Oly
PARSING  Ulfah Cafe
PARSING  71 Seventy One Coffee house
PARSING  Brother Joe Ikan Bakar & Seafood
PARSING  Cikong Power Mersing
PARSING  Dapur Papa
PARSING  Amma's Kitchen
PARSING  Mamat Burger Mersing
PARSING  Satay Nitar
PARSING  Ayam Gunting Mersing
PARSING  FNZ Corner
PARSING  Cheesy Popcorn Chicken (Kota Tinggi)
PARSING  Satay Pak Haron
PARSING  Kafe Witchery Ider Kota Tinggi 魔力點子
PARSING  Restoran Arsad @ Kota Jaya
PARSING  ANEKA SUP AMJ
PARSING  Amysha Village 3
PARSING  Restoran Al Riswan Bistro
PARSING  Alligator Cafe 鄂人泡泡奶茶
PARSING  Tadpole Cafe
PARSING  Green View Restaurant
PARSING  Restaurant Joo Shun
PARSING  Restoran Asri Rosmerah
PARSING  JIAYI Kota Tinggi 嘉壹哥打丁宜
PARSING  Restoran Shun Yii 順意海鮮飯店
PARSING  Mother Home Cook
PARSING  Food Court Sanggar Kota
PARSING

PARSING  Pasembor Corner
PARSING  Chor Char Koay Teow (Sg Siput)
PARSING  D'Villa Seafood Restaurant
PARSING  Restoran ABC Sheik
PARSING  Santai ARS Cafe
PARSING  Nasi Kandar AK Maju
PARSING  Kedai Makan Iela
PARSING  L & L CAFE
PARSING  Phongmun Vegetarian Restaurant
PARSING  Phongmun Restaurant
PARSING  Restoran Xin Ming Kee
PARSING  Sen Tao Sa Seafood
PARSING  Restoran Dayanis
PARSING  Melur Hati 
PARSING  Apit Burger
PARSING  Ravi's Corner TTS
PARSING  Yeop Helmi
PARSING  Selera Kampung Bendang Panjang
PARSING  Tailor Cafe
PARSING  Restoran Kaliani
PARSING  Liza Corner
PARSING  EZPIZZY POINT @  ARKED MARA KUALA KANGSAR
PARSING  Kafe Warisan
PARSING  Restoran Nor
PARSING  Essha Restaurant
PARSING  Restoran Pak Ude
PARSING  Restaurant Bilal Nasi Kandar Ayam Merah
PARSING  Dapur Faariqa
PARSING  Santai D' Firdaus
PARSING  Wani Cafe
PARSING  Ayam Penyet Kuala Kangsar
PARSING  Warisan Pak Ngah Laksa
PARSING  Take Me Home Chicken Rice
PARSING  Lem Kee Chicken Rice
PARSING  Restoran Sin L

PARSING  Kak Nik Nasi Kerabu
PARSING  THE PAMANS KITCHEN
PARSING  ABIM 
PARSING  D`aroma Restoran
PARSING  RESTORAN SRI MURNI TOMYAM
PARSING  DN 168 Restaurant
PARSING  Karlina Kiosks
PARSING  JF Restoran
PARSING  Poml Beverage 宝镁乐 @ Tangkak
PARSING  Tangkak Western House
PARSING  Corner Of The Lot
PARSING  Pizza Hut
PARSING  Siam Food House Restaurant
PARSING  DANISIHA SELERA PANTAI TIMUR
PARSING  Nanyang Kopitiam
PARSING  Restaurant Ruyi
PARSING  K Ten Ice Hut 
PARSING  SMELL COFFEE SHOP
PARSING  KFC
PARSING  Baker’s cottage
PARSING  Shanghai Tang 上海滩
PARSING  Chatto
PARSING  88 coffee shop
PARSING  Walls
PARSING  O Tea Tangkak
PARSING  SCC Coffee Shop
PARSING  Summer Ice Station
PARSING  Couture Cake Shop Tangkak
PARSING  KEDAI KOPI NORAFIZAH
PARSING  Savory Food & Drinks
PARSING  A Biao Restaurant(No.6 Stall)
PARSING  Rozaida Laksa Kola
PARSING  Restoran Sate Warisan Wak Buang
PARSING  Restoran Mee Bandung Tulang Rusuk
PARSING  Restoran ManMay
PARSING  Burger Sedap @Bukit gambir
PA

In [8]:
len(parsed)

9547

In [9]:
import json

with open('foodpanda-foods.json', 'w') as fopen:
    json.dump(restaurants, fopen)